In [ ]:
def get_config_dict():

    config_dict = {
        "workspace_id": notebookutils.runtime.context["currentWorkspaceId"],
        "lakehouse_name": "cms_lakehouse"
    }

    config_dict["lakehouse_id"] = notebookutils.lakehouse.get(config_dict["lakehouse_name"])['id']    
    return config_dict

In [ ]:
def get_full_abfss_path(workspace_id, lakehouse_id, dir):    
    full_abfss_path = f"abfss://{workspace_id}@onelake.dfs.fabric.microsoft.com/{lakehouse_id}/{dir}"
    return full_abfss_path


# plain pythfon code is used to write downloaded file to a non-defualt Lakehouse Files section
# and for that purpose 
def mount_path_return_local_path(full_abfss_path, mount_point):
    notebookutils.fs.mount(full_abfss_path, mount_point)
    return notebookutils.fs.getMountPath(mount_point)

In [ ]:
config_dict = get_config_dict()

In [ ]:
#create the sub-directory in Files folder where the CSV files will be downloaded
lakehouse_dir = "Files/cmsdemofiles"
lakehouse_dir_abfss_full_path = get_full_abfss_path(config_dict['workspace_id'], config_dict['lakehouse_id'], lakehouse_dir)

notebookutils.fs.mkdirs(lakehouse_dir_abfss_full_path)

print(f"lakehouse_dir: {lakehouse_dir}")
print(f"lakehouse_dir_abfss_full_path: {lakehouse_dir_abfss_full_path}")

In [ ]:
#local mount path is required for plain python to save files to Lakehouse file section
mount_point = "/mnt/lakehouse/" + config_dict["lakehouse_name"] + "/" + lakehouse_dir
lakehouse_dir_local_mount_path = mount_path_return_local_path(lakehouse_dir_abfss_full_path, mount_point)

print(f'mount point: {mount_point}')
print(f"lakehouse_dir_local_path: {lakehouse_dir_local_mount_path}")

In [ ]:
file_url = "https://github.com/isinghrana/fabric-samples-healthcare/raw/refs/heads/isr-auto1/analytics-bi-directlake-starschema/demoautomation/artifacts.zip"

In [ ]:
import requests

def download_binary_file(url, output_path):
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()  # Raise an error for bad status codes
        with open(output_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)
        print(f"File downloaded successfully to: {output_path}")
    except requests.exceptions.RequestException as e:
        print(f"Download failed: {e}")

# Example usage
#github_file_url = "https://github.com/user/repo/releases/download/v1.0.0/filename.bin"
#output_file_path = "artifacts.zip"
download_path = lakehouse_dir_local_mount_path + '/artifacts.zip'
download_binary_file(file_url, download_path)


In [ ]:
import zipfile
import os

def unzip_file(zip_path, extract_to):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    print(f"Extracted all contents to '{extract_to}'")

# Example usage

unzip_file(download_path, lakehouse_dir_local_mount_path)
